In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

# Definieer de parameters
input_shape = (224, 224, 3)  # Standaard inputgrootte voor het model
num_classes = 10  # Aantal klassen (kledingstukken en accessoires)

# Bouw het model (bijvoorbeeld een eenvoudige versie van YOLO)
def build_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    
    # Voorbeeld van een eenvoudig convolutieal netwerk
    x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    return keras.Model(inputs, outputs)

# Bouw het model
model = build_model(input_shape, num_classes)

# Compileer het model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Voorbewerking van de afbeeldingen (bijv. laden, normaliseren, augmentatie)
# Hier moet je je eigen code voor laden en voorbewerking van de afbeeldingen invoegen

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    # Functie om een afbeelding te laden en voor te bewerken
    
    # Controleer of het bestand bestaat
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")
    
    # Laden van de afbeelding
    image = cv2.imread(image_path)
    
    # Controleer of het laden succesvol was
    if image is None:
        raise IOError(f"Unable to load image: {image_path}")
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Omzetten naar RGB (indien nodig)
    
    # Resize van de afbeelding naar de gewenste grootte
    image = cv2.resize(image, target_size)
    
    # Normaliseren van de pixelwaarden naar het bereik [0, 1]
    image = image / 255.0
    
    return image

def load_and_preprocess_labels(label_path, num_classes=10, image_size=(224, 224)):
    # Functie om labels te laden en voor te bewerken
    
    # Lees het labelbestand
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    labels = []
    
    for line in lines:
        # Split de regel in componenten
        parts = line.strip().split()
        
        # Extract klasse en bounding box coördinaten
        class_id = int(parts[0])
        center_x = float(parts[1])
        center_y = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # Converteer naar absolute coördinaten
        x1 = (center_x - width / 2) * image_size[1]
        y1 = (center_y - height / 2) * image_size[0]
        x2 = (center_x + width / 2) * image_size[1]
        y2 = (center_y + height / 2) * image_size[0]
        
        # Voeg de label toe in het juiste formaat voor het model
        labels.append([x1, y1, x2, y2, class_id])
    
    return np.array(labels)

# Train het model
# Voorbeeldcode voor het laden van afbeeldingen en labels
# Dit moet worden aangepast aan de specifieke structuur van je dataset

image_paths = "Train/images"  # Lijst van paden naar afbeeldingen
label_paths = "Train/labels"  # Lijst van paden naar labelbestanden

num_epochs = 15

# Voorbeeld van een training loop (de echte implementatie hangt af van je datasetstructuur)
for epoch in range(num_epochs):
    for image_path, label_path in zip(image_paths, label_paths):
        # Laad afbeelding en labels
        image = load_and_preprocess_image(image_path)
        labels = load_and_preprocess_labels(label_path)
        
        # Train het model op de batch
        model.train_on_batch(image, labels)

# Save the model
model.save('object_detection_model.h5')


FileNotFoundError: Image file not found: T

In [6]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Definieer het pad naar de afbeeldingen en annotaties
# image_dir = '/content/drive/My Drive/Train/images'
# labels_dir = '/content/drive/My Drive/Train/labels'

image_dir = 'Train/images'
labels_dir = 'Train/labels'


# Functie om annotaties in te lezen
def read_labels(lables_path):
    with open(labels_path, 'r') as file:
        lines = file.readlines()

    labels = []
    for line in lines:
        parts = line.strip().split()
        item_klasse = int(parts[0])
        x_center_norm = float(parts[1])
        y_center_norm = float(parts[2])
        breedte_norm = float(parts[3])
        hoogte_norm = float(parts[4])
        labels.append((item_klasse, x_center_norm, y_center_norm, breedte_norm, hoogte_norm))

    return labels

# Functie om genormaliseerde waarden om te zetten naar pixelwaarden
def denormalize_bbox(image_width, image_height, bbox):
    item_class, x_center_norm, y_center_norm, width_norm, height_norm = bbox
    x_center = x_center_norm * image_width
    y_center = y_center_norm * image_height
    width = width_norm * image_width
    height = height_norm * image_height
    x = x_center - (width / 2)
    y = y_center - (height / 2)
    return item_class, x, y, width, height

# Lijst van item klassen
item_classes = ["sunglass", "hat", "jacket", "shirt", "pants", "shorts", "skirt", "dress", "bag", "shoe"]

# Kies een willekeurige afbeelding en de bijbehorende annotaties om te visualiseren
image_files = os.listdir(image_dir)
sample_image_file = image_files[1]  # Je kunt willekeurig kiezen of een andere manier om te selecteren

image_path = os.path.join(image_dir, sample_image_file)
labels_path = os.path.join(labels_dir, sample_image_file.replace('.jpg', '.txt'))

# Lees de afbeelding en de annotaties
image = Image.open(image_path)
image_width, image_height = image.size
labels = read_labels(labels_path)

# Zet genormaliseerde bounding boxes om naar pixelwaarden
bboxes = [denormalize_bbox(image_width, image_height, bbox) for bbox in labels]

In [7]:
import os
import cv2
import numpy as np

# Functie om annotaties in te lezen
def read_labels(labels_path):
    boxes = []
    with open(labels_path, 'r') as file:
        for line in file.readlines():
            parts = line.strip().split()
            item_class = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            boxes.append([item_class, x_center, y_center, width, height])
    return np.array(boxes)

# Functie om afbeeldingen en labels in te lezen
def load_data(image_dir, labels_dir):
    image_files = os.listdir(image_dir)
    images = []
    targets = []

    for img_name in image_files:
        img_path = os.path.join(image_dir, img_name)
        label_path = os.path.join(labels_dir, img_name.replace('.jpg', '.txt'))

        image = cv2.imread(img_path)
        boxes = [denormalize_bbox(image_width, image_height, bbox) for bbox in labels]

        images.append(image)
        targets.append(boxes)

    return images, targets

# Functie om afbeeldingen voor te bewerken
def preprocess_images(images):
    resized_images = [cv2.resize(image, (416, 416)) for image in images]
    normalized_images = [image.astype(np.float32) / 255.0 for image in resized_images]
    return normalized_images

images, targets = load_data(image_dir, labels_dir)
images = preprocess_images(images)



In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

# Definieer de parameters
input_shape = (224, 224, 3)  # Standaard inputgrootte voor het model
num_classes = 10  # Aantal klassen (kledingstukken en accessoires)

# Bouw het model (bijvoorbeeld een eenvoudige versie van YOLO)
def build_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    
    # Voorbeeld van een eenvoudig convolutieal netwerk
    x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    return keras.Model(inputs, outputs)

# Bouw het model
model = build_model(input_shape, num_classes)

# Compileer het model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



# Train het model
# Voorbeeldcode voor het laden van afbeeldingen en labels
# Dit moet worden aangepast aan de specifieke structuur van je dataset
import numpy as np
import tensorflow as tf
from tensorflow import keras



# Example data (replace with your actual data loading process)
image_batch = images
labels_batch = labels
# Ensure data types are correct
# Ensure data types are correct
image_batch = image_batch.astype(np.float32)

# Convert labels_batch to numpy array if it's not already
if not isinstance(labels_batch, np.ndarray):
    labels_batch = np.array(labels_batch, dtype=np.int32)

# Verify shapes before training
print(image_batch.shape)  # Should print (32, 64, 64, 3)
print(labels_batch.shape)  # Should print (32,)



# Verify shapes before training
print(image_batch.shape)  # Should print (32, 64, 64, 3)
print(labels_batch.shape)  # Should print (32,)

# Train on batch
model.train_on_batch(image_batch, labels_batch)




# Save the model
model.save('object_detection_model.h5')


TypeError: 'numpy.ndarray' object is not callable

In [12]:
import numpy as np

image_batch = images
labels_batch = labels
# Ensure data types are correct
# Ensure data types are correct
image_batch = image_batch.astype(np.float32)

# Convert labels_batch to numpy array if it's not already
if not isinstance(labels_batch, np.ndarray):
    labels_batch = np.array(labels_batch, dtype=np.int32)

# Print contents
print("Image batch:")
print(image_batch)

print("\nLabels batch:")
print(labels_batch)


Image batch:
[[[0.9019608  0.8980392  0.9137255 ]
  [0.8980392  0.89411765 0.9098039 ]
  [0.8901961  0.8862745  0.9019608 ]
  ...
  [0.8901961  0.8862745  0.9019608 ]
  [0.8784314  0.8745098  0.8901961 ]
  [0.87058824 0.8666667  0.88235295]]

 [[0.9019608  0.8980392  0.9137255 ]
  [0.8980392  0.89411765 0.9098039 ]
  [0.8901961  0.8862745  0.9019608 ]
  ...
  [0.8862745  0.88235295 0.8980392 ]
  [0.8745098  0.87058824 0.8862745 ]
  [0.8627451  0.85882354 0.8745098 ]]

 [[0.9019608  0.8980392  0.9137255 ]
  [0.8980392  0.89411765 0.9098039 ]
  [0.8901961  0.8862745  0.9019608 ]
  ...
  [0.88235295 0.8784314  0.89411765]
  [0.87058824 0.8666667  0.88235295]
  [0.8627451  0.85882354 0.8745098 ]]

 ...

 [[0.27058825 0.2784314  0.32156864]
  [0.26666668 0.27450982 0.31764707]
  [0.25882354 0.26666668 0.30980393]
  ...
  [0.34117648 0.34509805 0.38431373]
  [0.35686275 0.36078432 0.4       ]
  [0.37254903 0.3764706  0.41568628]]

 [[0.25882354 0.26666668 0.30980393]
  [0.25490198 0.25882354